In [1]:
%load_ext autoreload
%autoreload 2
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
import matplotlib.pyplot as plt
from pyspark.sql import functions as F
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
import utils
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, count
from pyspark.sql.types import ArrayType, StringType, BooleanType, IntegerType, DoubleType

label_column = 'LOS'

In [2]:
spark = SparkSession.builder \
    .appName("Intensive Care Unit Data Analysis") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "6g") \
    .config("spark.memory.fraction", "0.9") \
    .config("spark.memory.storageFraction", "0.5") \
    .getOrCreate()

# check spark configs to only errors:

spark.sparkContext.setLogLevel("ERROR")
spark.conf.set("spark.sql.shuffle.partitions", "500")
spark.conf.set("spark.sql.debug.maxToStringFields", "1000")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/08 12:04:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Read the datasets
df_admissions = spark.read.csv("data/ADMISSIONS.csv", header=True, inferSchema=True).drop("ROW_ID")
df_diagnoses = spark.read.csv("data/DIAGNOSES_ICD.csv", header=True, inferSchema=True).drop("ROW_ID")
df_icustays = spark.read.csv("data/ICUSTAYS.csv", header=True, inferSchema=True).drop("ROW_ID")
df_patients = spark.read.csv("data/PATIENTS.csv", header=True, inferSchema=True).drop("ROW_ID")
# df_chartevents = spark.read.csv("data/CHARTEVENTS.csv", header=True, inferSchema=True).drop("ROW_ID")

In [4]:
df = df_patients.join(df_admissions, df_patients["SUBJECT_ID"] == df_admissions["SUBJECT_ID"], how="left").drop(df_admissions["SUBJECT_ID"])
df = df.join(df_icustays, df["HADM_ID"] == df_icustays["HADM_ID"], how="left").drop(df_icustays["SUBJECT_ID"]).drop(df_icustays["HADM_ID"])
df = df.join(df_diagnoses, df["SUBJECT_ID"] == df_diagnoses["SUBJECT_ID"], how="left").drop(df_diagnoses["SUBJECT_ID"]).drop(df_diagnoses["HADM_ID"])

In [5]:
df.show()

+----------+------+-------------------+----+--------+-------+-----------+-------+-------------------+-------------------+---------+--------------+--------------------+------------------+---------+--------+--------+--------------+---------+-------------------+-------------------+--------------------+--------------------+--------------------+----------+--------+--------------+-------------+------------+-----------+-------------------+-------------------+------+-------+---------+
|SUBJECT_ID|GENDER|                DOB| DOD|DOD_HOSP|DOD_SSN|EXPIRE_FLAG|HADM_ID|          ADMITTIME|          DISCHTIME|DEATHTIME|ADMISSION_TYPE|  ADMISSION_LOCATION|DISCHARGE_LOCATION|INSURANCE|LANGUAGE|RELIGION|MARITAL_STATUS|ETHNICITY|          EDREGTIME|          EDOUTTIME|           DIAGNOSIS|HOSPITAL_EXPIRE_FLAG|HAS_CHARTEVENTS_DATA|ICUSTAY_ID|DBSOURCE|FIRST_CAREUNIT|LAST_CAREUNIT|FIRST_WARDID|LAST_WARDID|             INTIME|            OUTTIME|   LOS|SEQ_NUM|ICD9_CODE|
+----------+------+-----------------

Check for missing values in the dataset.

In [6]:
utils.print_missing_value_counts(df)

-RECORD 0-----------------------
 SUBJECT_ID           | 0       
 GENDER               | 0       
 DOB                  | 0       
 DOD                  | 697342  
 DOD_HOSP             | 892628  
 DOD_SSN              | 839693  
 EXPIRE_FLAG          | 0       
 HADM_ID              | 0       
 ADMITTIME            | 0       
 DISCHTIME            | 0       
 DEATHTIME            | 1359709 
 ADMISSION_TYPE       | 0       
 ADMISSION_LOCATION   | 0       
 DISCHARGE_LOCATION   | 0       
 INSURANCE            | 0       
 LANGUAGE             | 379832  
 RELIGION             | 3721    
 MARITAL_STATUS       | 78080   
 ETHNICITY            | 0       
 EDREGTIME            | 476403  
 EDOUTTIME            | 476403  
 DIAGNOSIS            | 747     
 HOSPITAL_EXPIRE_FLAG | 0       
 HAS_CHARTEVENTS_DATA | 0       
 ICUSTAY_ID           | 31975   
 DBSOURCE             | 31975   
 FIRST_CAREUNIT       | 31975   
 LAST_CAREUNIT        | 31975   
 FIRST_WARDID         | 31975   
 LAST_WARD

# Feature Engineering

Let's create a column called 'AGE'

In [7]:
# Ensure 'DOB' and 'ADMITTIME' are in the correct date format if not already
df = df.withColumn('DOB', F.to_date('DOB'))
df = df.withColumn('ADMITTIME', F.to_date('ADMITTIME'))

# Create the 'AGE' column by calculating the difference in years between 'ADMITTIME' and 'DOB'
df = df.withColumn('AGE', F.expr("floor(months_between(ADMITTIME, DOB) / 12)"))

It only matters if the patient is dead or not, not the date of death, so we can drop the date of death columns. And we also drop other death-related columns.

In [8]:
df=df.drop("DOB").drop("DOD").drop("DOD_SSN").drop("EXPIRE_FLAG").drop("DEATHTIME")
df = df.withColumn("DOD_HOSP", F.when(F.col("DOD_HOSP").isNull(), 0).otherwise(1))
df = df.withColumnRenamed("DOD_HOSP", "DIED")
df = df.withColumnRenamed("ICD9_CODE","DISEASES_CODE")

Here we will drop the columns that are not useful for the analysis.

In [9]:
columns_to_remove = [
    "ADMITTIME", "DISCHTIME", "EDREGTIME", "EDOUTTIME", "HOSPITAL_EXPIRE_FLAG",
    "INTIME", "OUTTIME","LANGUAGE","DISCHARGE_LOCATION",
    "ICUSTAY_ID", "SEQ_NUM","HAS_CHARTEVENTS_DATA","DBSOURCE"
]

df = df.drop(*columns_to_remove)

The ethnicity column has too many unique values. We can group them into a few categories.

In [10]:
# Define the transformation using multiple chained when conditions
df = df.withColumn("ETHNICITY",
    F.when(F.col("ETHNICITY").isin('AMERICAN INDIAN/ALASKA NATIVE', 'AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE'),
           'American Indian/Alaska Native')
    .when(F.col("ETHNICITY").isin('ASIAN', 'ASIAN - ASIAN INDIAN', 'ASIAN - CAMBODIAN', 'ASIAN - CHINESE', 'ASIAN - FILIPINO', 'ASIAN - JAPANESE', 'ASIAN - KOREAN', 'ASIAN - OTHER', 'ASIAN - THAI', 'ASIAN - VIETNAMESE'),
          'Asian')
    .when(F.col("ETHNICITY").isin('BLACK/AFRICAN', 'BLACK/AFRICAN AMERICAN', 'BLACK/CAPE VERDEAN', 'BLACK/HAITIAN'),
          'Black')
    .when(F.col("ETHNICITY").isin('HISPANIC OR LATINO', 'HISPANIC/LATINO - CENTRAL AMERICAN (OTHER)', 'HISPANIC/LATINO - COLOMBIAN', 'HISPANIC/LATINO - CUBAN', 'HISPANIC/LATINO - DOMINICAN', 'HISPANIC/LATINO - GUATEMALAN', 'HISPANIC/LATINO - HONDURAN', 'HISPANIC/LATINO - MEXICAN', 'HISPANIC/LATINO - PUERTO RICAN', 'HISPANIC/LATINO - SALVADORAN'),
          'Hispanic/Latino')
    .when(F.col("ETHNICITY").isin('MIDDLE EASTERN'),
          'Middle Eastern')
    .when(F.col("ETHNICITY").isin('NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER'),
          'Pacific Islander')
    .when(F.col("ETHNICITY").isin('WHITE', 'WHITE - BRAZILIAN', 'WHITE - EASTERN EUROPEAN', 'WHITE - OTHER EUROPEAN', 'WHITE - RUSSIAN', 'PORTUGUESE'),
          'White')
    .when(F.col("ETHNICITY").isin('CARIBBEAN ISLAND', 'SOUTH AMERICAN'),
          'Caribbean/South American')
    .when(F.col("ETHNICITY").isin('MULTI RACE ETHNICITY'),
          'Multi-Race')
    .when(F.col("ETHNICITY").isin('OTHER'),
          'Other')
    .otherwise('NO DATA REGISTERED')
)

Now we can aggregate the data by primary keys and collect the remaining columns into lists.

In [11]:
# Define your primary key columns
primary_key_columns = ["SUBJECT_ID", "HADM_ID"]

# Identify the remaining columns to be grouped
remaining_columns = [col for col in df.columns if col not in primary_key_columns]

# Group by the primary key columns and aggregate the remaining columns into lists
df = df.groupBy(primary_key_columns).agg(*(F.collect_list(col).alias(col) for col in remaining_columns))

df=df.drop("HADM_ID")

Now we can transform the columns that contain lists of values. If all values in the list are the same, we can replace the list with a single value. If the list is empty, we can replace it with a default value.

In [12]:
df.show()

+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|SUBJECT_ID|              GENDER|                DIED|      ADMISSION_TYPE|  ADMISSION_LOCATION|           INSURANCE|            RELIGION|      MARITAL_STATUS|           ETHNICITY|           DIAGNOSIS|      FIRST_CAREUNIT|       LAST_CAREUNIT|        FIRST_WARDID|         LAST_WARDID|                 LOS|       DISEASES_CODE|                 AGE|
+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------

In [13]:
replace_empty_list_udf = udf(utils.replace_empty_list, ArrayType(StringType()))
transform_list_udf = udf(utils.transform_list, ArrayType(StringType()))
handle_list_udf = udf(utils.handle_list, StringType())
empty_list_udf = F.udf(lambda col: col == [], BooleanType())

for column in df.columns:    
    if isinstance(df.schema[column].dataType, ArrayType):
        if df.filter(empty_list_udf(F.col(column))).count() > 0:
            df = df.withColumn(column, replace_empty_list_udf(col(column)))
            
        df = df.withColumn(column, transform_list_udf(col(column)))
        if len(set(df.select(column).first()[0])) == 1:
            df = df.withColumn(column, handle_list_udf(df[column]))

Check for missing values again.

In [14]:
utils.print_missing_value_counts(df)

ERROR:root:KeyboardInterrupt while sending command.                 (1 + 7) / 8]
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/bigdata/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/bigdata/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/bigdata/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt
Exception ignored in: <function JavaObject.__init__.<locals>.<lambda> at 0x124017560>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/bigdata/lib/python3.11/site-packages/py4j

KeyboardInterrupt: 

In [ ]:
df.show()

Now let's add the biggest table to the dataframe.

In [ ]:
# df = df.join(df_chartevents,df["SUBJECT_ID"] == df_chartevents["SUBJECT_ID"], how="left").drop(df_chartevents["SUBJECT_ID"]).drop(df_chartevents["HADM_ID"]).drop(df_chartevents["ICUSTAY_ID"])

# Graphical Analysis

Uncomment the following code to load the dataset from a CSV file.

In [ ]:
# Load the CSV file
#df = spark.read.csv("dataset.csv", header=True, inferSchema=True)

# Convert the comma-separated string back to an array
#df_with_array = df.withColumn("DISEASES_CODE", split(col("DISEASES_CODE"), ","))

In [ ]:
# Assuming `df` is your PySpark DataFrame
df = df.withColumn("DIED", df["DIED"].cast("integer"))

# Plotting death counts by gender
utils.plot_graph(df, 'GENDER', 'DIED', F.sum, 'Death Counts by Gender', 'Gender', 'Number of Deaths')

## Unique Value Count of marital status

In [ ]:
utils.plot_graph(df, 'MARITAL_STATUS', 'MARITAL_STATUS', F.count, 'Count of Marital Status', 'Marital Status', 'Count')

## Unique Value Count of Admission Types

In [ ]:
utils.plot_graph(df, 'ADMISSION_TYPE', 'ADMISSION_TYPE', F.count, 'Count of Admission Types', 'Admission Type', 'Count')

## Distribution of Lethality by Religion

The 1º and 2º deadliest religions are not shown in the graph because they are outliers.

In [ ]:
# Calculate death rate per religion
death_rate_per_religion = df.groupBy('RELIGION')\
    .agg(
        F.sum('DIED').alias('Deaths'), 
        F.count('DIED').alias('Total')
    )\
    .withColumn('Death Rate', (F.col('Deaths') / F.col('Total')) * 100)\
    .orderBy(F.col('Death Rate').desc())

religions = death_rate_per_religion.rdd.zipWithIndex().filter(lambda x: x[1] in [2,3,4,5,6,7]).map(lambda x: x[0]).toDF()

utils.plot_graph(religions, 'RELIGION', 'Death Rate', F.first, 'Death Rates for Religions Ranked', 'Religion', 'Death Rate (%)')

## Distribution of Lethality by Ethnicity

In [ ]:
filtered_df = df.filter(df['ETHNICITY'] != 'NO DATA REGISTERED')

# Calculate death rate per ethnicity
death_rate_per_ethnicity = filtered_df.groupBy('ETHNICITY')\
    .agg(
        F.sum('DIED').alias('Deaths'), 
        F.count('DIED').alias('Total')
    )\
    .withColumn('Death Rate', (F.col('Deaths') / F.col('Total')) * 100)\
    .orderBy(F.col('Death Rate').desc())\
    .limit(5)

utils.plot_graph(death_rate_per_ethnicity, 'ETHNICITY', 'Death Rate', F.first, 'Top 5 Deadliest Ethnicities by Death Rate', 'Ethnicity', 'Death Rate (%)')

## Distribution of Length of Stay by Age

Uncomment the following code to save the dataset to a CSV file.

In [15]:
# Assuming 'df' is your DataFrame
df_modified = df.withColumn("DISEASES_CODE", F.concat_ws(",", col("DISEASES_CODE")))

df_modified.write.csv("dataset.csv", header=True, mode="overwrite", sep=",")

# Pre processing

In [ ]:
df.show()

In [ ]:
df = df.withColumn("GENDER", F.when(col("GENDER") == "M", 0).otherwise(1))
check_presence_udf = udf(lambda category_list, value: 1 if value in category_list else 0, IntegerType())

# Find unique categories and generate columns for each category
for category in df.select("DISEASES_CODE").rdd.flatMap(lambda x: x[0]).distinct().collect():
    df = df.withColumn(f'DISEASE_CODE_{category}', check_presence_udf(col("DISEASES_CODE"), F.lit(category)))
    
df = df.drop("DISEASES_CODE")

In [ ]:
all_columns = df.columns
feature_columns = [col for col in all_columns if col not in ['SUBJECT_ID', 'LOS']]

# Manually specified categorical and numerical features
categorical_features = ['ADMISSION_TYPE', 'ADMISSION_LOCATION', 'INSURANCE', 'RELIGION',
                        'MARITAL_STATUS', 'ETHNICITY', 'DIAGNOSIS', 'FIRST_CAREUNIT', 'LAST_CAREUNIT']
numerical_features = [col for col in feature_columns if col not in categorical_features and col not in ['SUBJECT_ID', 'LOS']]

# Cast string numerical features to float
df = df.withColumn("AGE", col("AGE").cast("float"))
df = df.withColumn("FIRST_WARDID", col("FIRST_WARDID").cast("float"))
df = df.withColumn("LAST_WARDID", col("LAST_WARDID").cast("float"))
df = df.withColumn("LOS", col("LOS").cast("float"))

In [ ]:
df.show()

In [ ]:
# Stages in the pipeline
stages = []

# Indexing and encoding categorical features
for categoricalCol in categorical_features:
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index", handleInvalid="keep")
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "ClassVec"])
    stages += [stringIndexer, encoder]

# Assemble all the features along with the encoded categorical features
assemblerInputs = [c + "ClassVec" for c in categorical_features] + numerical_features
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features", handleInvalid="keep")
stages += [assembler]

# Pipeline: This will ensure all stages are applied in sequence
pipeline = Pipeline(stages=stages)
pipelineModel = pipeline.fit(df)
df_transformed = pipelineModel.transform(df)

In [ ]:
# Assume subject_ids have been collected as before
subject_ids = [row['SUBJECT_ID'] for row in df_transformed.select("SUBJECT_ID").distinct().collect()]
split_index = int(len(subject_ids) * 0.8)

train_ids = set(subject_ids[:split_index])
test_ids = set(subject_ids[split_index:])

# Directly filter the DataFrame using the list
train_data = df_transformed.filter(col("SUBJECT_ID").isin(train_ids))
test_data = df_transformed.filter(col("SUBJECT_ID").isin(test_ids))

# Check the count of entries in each dataset
print("Entries in train_data:", train_data.count())
print("Entries in test_data:", test_data.count())

In [ ]:
train_data.show()

In [ ]:
test_data.show()

Here we are making sure that there isn't simultaneously a patient in the test set and train set.

# Prediction

In [ ]:
def calculate_accuracy(predictions, label_column):
    # Calculate the Mean Absolute Error (MAE)
    mae = predictions.withColumn('abs_error', abs(col(label_column) - col("prediction"))) \
                     .select(F.avg('abs_error')).first()[0]

    # Convert MAE to a pseudo-accuracy metric (assuming MAE is less than 100)
    accuracy = 100 - mae  # This assumes that lower error corresponds to higher accuracy

    return accuracy

## Linear Regression

In [ ]:
param = {
    'maxIter': 10,
    'regParam': 0.3,
    'elasticNetParam': 0.8
}

# Define and fit the Linear Regression model on the training set
lr = LinearRegression(featuresCol='features', labelCol=label_column, **param)
lr_model = lr.fit(train_data)

## Random Forest Regression

In [ ]:
param = {
    'maxDepth': 5,
    'maxBins': 32,
    'minInstancesPerNode': 1,
    'minInfoGain': 0.0,
    'maxMemoryInMB': 256,
    'cacheNodeIds': False,
    'checkpointInterval': 10,
    'impurity': 'variance',
    'featureSubsetStrategy': 'auto',
    'subsamplingRate': 1.0,
    'seed': None,
    'numTrees': 20,
}

# Define and fit the Random Forest model on the training set
rf = RandomForestRegressor(featuresCol='features', labelCol=label_column, **param)
rf_model = rf.fit(train_data)

## Model Evaluation

In [ ]:
# Evaluate models using RMSE
evaluator = RegressionEvaluator(labelCol=label_column, predictionCol="prediction", metricName="rmse")

# Generate predictions on both training and testing datasets
lr_train_predictions = lr_model.transform(train_data)
lr_test_predictions = lr_model.transform(test_data)
rf_train_predictions = rf_model.transform(train_data)
rf_test_predictions = rf_model.transform(test_data)

# Calculate RMSE for Linear Regression
lr_train_rmse = evaluator.evaluate(lr_train_predictions)
lr_test_rmse = evaluator.evaluate(lr_test_predictions)

# Calculate RMSE for Random Forest
rf_train_rmse = evaluator.evaluate(rf_train_predictions)
rf_test_rmse = evaluator.evaluate(rf_test_predictions)

# Print RMSE results
print(f"Linear Regression - Train RMSE: {lr_train_rmse}, Test RMSE: {lr_test_rmse}")
print(f"Random Forest - Train RMSE: {rf_train_rmse}, Test RMSE: {rf_test_rmse}")